# Script for web scrapping cwc real time data
### import libraries 

In [ ]:
import requests
import json
import pandas as pd
import datetime
from datetime import date
# import openpyxl
# from openpyxl.utils.dataframe import dataframe_to_rows
import os

### set file export path, file suffix and start and end date

In [ ]:
file_path_exp="F:\\phd_work\\cwc_real_time_data\\updated_real_time\\data_files\\2020_2024\\"

file_suffix='2020_2024.xlsx'

sdate='2024-01-01' # yyyy-mm-dd
edate='2024-12-31' # yyyy-mm-dd

In [ ]:
url = "https://ffs.india-water.gov.in/iam/api/new-entry-data/specification/sorted"
df_code=pd.read_excel('F:\\phd_work\\cwc_real_time_data\\updated_real_time\\data_files\\2020_updated\\name-code.xlsx',index_col=0)
df_code['code_id']=df_code.index
df_code=df_code.reset_index(drop=True)
datetime_format = '%Y-%m-%dT%H:%M:%S'

for index,df_code_ext in df_code.iloc[:].iterrows():
    st_code=df_code_ext['code_id']
    name_station=df_code_ext['name']
    print('extracting data for ::',name_station)
    try:
        params = {
            "sort-criteria": "%7B%22sortOrderDtos%22:%5B%7B%22sortDirection%22:%22ASC%22,%22field%22:%22id.dataTime%22%7D%5D%7D",
            "specification": f"%7B%22where%22:%7B%22where%22:%7B%22where%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.stationCode%22,%22operator%22:%22eq%22,%22value%22:%22{st_code}%22%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.datatypeCode%22,%22operator%22:%22eq%22,%22value%22:%22HHS%22%7D%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22dataValue%22,%22operator%22:%22null%22,%22value%22:%22false%22%7D%7D%7D,%22and%22:%7B%22expression%22:%7B%22valueIsRelationField%22:false,%22fieldName%22:%22id.dataTime%22,%22operator%22:%22btn%22,%22value%22:%22{sdate}T14:05:31.255,{edate}T14:05:31.255%22%7D%7D%7D"
        }

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        response = requests.get(url, params=params, headers=headers)
        data = response.json()

        df = pd.DataFrame()
        fl=[]
        for j in data[:]:
            try:
                ml=[j['stationCode'],datetime.datetime.strptime(j['id']['dataTime'],datetime_format),name_station,j['dataValue']]
                fl.append(ml)
            except:
                pass
        if fl:
            print('recieved data for :: ',name_station)
            fl=sorted(fl)
            df=pd.DataFrame(fl)
            df.columns=['code','time','name','WSE']
            df= df.drop(columns=['Unnamed: 0'])
            df.to_excel(file_path_new+name_station+new_file_suffix) # new file end
            print('Data exported for station ',name_station)
        else:
            print('Data export failed for XXXXXXXXXXXXXXXXX',name_station)
    except:
        print('Data not found for the station ',name_station)